# 如何传递私有状态

通常，您可能希望节点能够相互传递状态，而这不应成为图的主模式的一部分。这通常很有用，因为可能有些信息不需要作为输入/输出（因此在主模式中没有真正意义），但需要作为中间工作逻辑的一部分。

让我们看一下下面的例子。在此示例中，我们将创建一个 RAG 管道：
1. 接受用户问题
2. 使用 LLM 生成搜索查询
3. 检索生成的查询的文档
4. 根据这些文档生成最终答案

我们将为每个步骤都有一个单独的节点。整体状态上我们只有 `question` 和 `answer`。但是，我们需要 `search_query` 和 `documents` 的单独状态 - 我们将通过在每个相关节点上定义 `input` 注释来将它们作为私有状态密钥传递。

让我们看一个例子！

In [1]:
import { Annotation, StateGraph } from "@langchain/langgraph";


// 图表的整体状态
const OverallStateAnnotation = Annotation.Root({
  question: Annotation<string>,
  answer: Annotation<string>,
});

// 这是生成查询的节点将返回的内容
const QueryOutputAnnotation = Annotation.Root({
  query: Annotation<string>,
});

// 这是检索文档的节点将返回的内容
const DocumentOutputAnnotation = Annotation.Root({
  docs: Annotation<string[]>,
});

// 这是检索文档的节点将返回的内容
const GenerateOutputAnnotation = Annotation.Root({
  ...OverallStateAnnotation.spec,
  ...DocumentOutputAnnotation.spec
});

// 生成查询的节点
const generateQuery = async (state: typeof OverallStateAnnotation.State) => {
  // 用真正的逻辑替换它
  return {
    query: state.question + " rephrased as a query!",
  };
};

// 检索文档的节点
const retrieveDocuments = async (state: typeof QueryOutputAnnotation.State) => {
  // 用真正的逻辑替换它
  return {
    docs: [state.query, "some random document"],
  };
};

// 生成答案的节点
const generate = async (state: typeof GenerateOutputAnnotation.State) => {
  return {
    answer: state.docs.concat([state.question]).join("\n\n"),
  };
};

const graph = new StateGraph(OverallStateAnnotation)
  .addNode("generate_query", generateQuery)
  .addNode("retrieve_documents", retrieveDocuments, { input: QueryOutputAnnotation })
  .addNode("generate", generate, { input: GenerateOutputAnnotation })
  .addEdge("__start__", "generate_query")
  .addEdge("generate_query", "retrieve_documents")
  .addEdge("retrieve_documents", "generate")
  .compile();

await graph.invoke({
  question: "How are you?",
});

{
  question: 'How are you?',
  answer: 'How are you? rephrased as a query!\n\nsome random document\n\nHow are you?'
}


上面，输入中的原始 `question` 值已被保留，但 `generate_query` 节点对其进行了重新表述，`retrieve_documents` 节点添加了 `"some random document"`，最后 `generate` 节点将状态中的 `docs` 与原始问题组合起来创建 `answer`。由传递到各个节点的 `input` 注释填充的中间步骤不会出现在最终输出中。